In [8]:
!python --version
%pip install panda
%pip install numpy
%pip install scikit-learn
%pip install matplotlib
%pip install plot_keras_history
%pip install tensorflow
%pip install keras-tuner
%pip install xlrd
%pip install openpyxl

Python 3.11.5
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [13]:
import time
import datetime

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from plot_keras_history import plot_history
import sklearn.metrics as metrics
import os as os

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.python.client import device_lib
import keras_tuner as kerastuner
from keras_tuner.tuners import RandomSearch


print(tf.__version__)

def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

#Se va a habilitar la dedicacion dinamica de memoria para que la GPU vaya asignando recursos al proceso conforme los vaya necesitando
#physical_devices =  get_available_devices() #Obtener la lista de GPU's instaladas en la maquina

#print(get_available_devices())

#tf.config.experimental.set_memory_growth(physical_devices[0], True) #Permitir la reserva dinamica de memoria en la GPU con id=0

if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
#print(tf.config.list_physical_devices('GPU'))
print(tf.test.is_built_with_cuda())



# Historico de datos para realizar la predicción
W = 730
# Insantes de tiempo futuros a predecir
H = 10

# Porcentaje del conjunto de test
test_size = 0.3
# Porcentaje del conjunto de validacion
val_size = 0.3


# Mejores modelos que se quieren obtener
numBestModels = 1
# Establecer iteraciones (max_trials) y ejecuciones por cada iteracion (executions_per_trial)
# max_trials = Total number of trials (model configurations) to test at most. Note that the oracle may interrupt the search before max_trial models have been tested if the search space has been exhausted.
trials = 10
executions = 20
# Establecer objetivo (Name of model metric to minimize or maximize, e.g. "val_accuracy"). el "val_" hace referencia a que se coge la métrica en el subconjunto de validación

obje = kerastuner.Objective('val_mean_absolute_percentage_error', 'min')


# Epocas
searchEpochs = 2
epch = 10
# Tamaño del batch
batch = 1024
# Establecer minimo y maximo de unidades, el salto en el que pueden cambiar y el valor por defecto
minUnits = 50
maxUnits = 300
stepsUnits = 25
defaultUnits = 50
# Establecer minimo y maximo de capas y el valor por defecto
minLayers = 1
maxLayers = 10
defaultLayers = 3
# Establecer Dropout
minDropout = 0
maxDropout = .4
defaultDropout = .2
# Establecer learning rate
lr = [0.0, 1e-2, 1e-3, 1e-4]

2.15.0
Please install GPU version of TF
Num GPUs Available:  0
False


In [10]:
df = pd.read_excel("C:/Users/hecto/Documents/Master/TFM/tfm-renewable-energy-deep-learning/2015_30min.xlsx", header=None, engine='openpyxl')[0]

# Data preprocessing: each row will contain the 10 measures for each day , and the 10 measures for the following day
X = pd.DataFrame(np.array(df).reshape(-1, 10), columns=["col_{}".format(i) for i in range(0, 10)])

Y = pd.DataFrame.copy(X)

Y.columns = ["col_{}".format(i) for i in range(11, 21)]
Y = Y.drop(0)
Y = Y.reset_index(drop=True)
Y.loc[len(Y)] = np.zeros(10)

dfPreproccessed = pd.concat([X, Y], axis=1)

print("DataFrame Preproccessed:")
print(dfPreproccessed)
print("---------------------------------------------")

pd.set_option('display.max_columns', dfPreproccessed.shape[1])
print("Head DataFrame Preprocessed\n " + str(dfPreproccessed.head()))
print("---------------------------------------------")

# Split the data into training and validation sets
xTrain, xTest, yTrain, yTest = train_test_split(X, Y, test_size = test_size, random_state = 0, shuffle=False)
xTrain, xVal, yTrain, yVal = train_test_split(xTrain, yTrain, test_size = val_size, random_state = 0, shuffle=False)

# Show subsets shapes
print("Shapes:")
print("xTrain:\t"+str(xTrain.shape))
print("yTrain:\t"+str(yTrain.shape))
print("xVal:\t"+str(xVal.shape))
print("yVal:\t"+str(yVal.shape))
print("xTest:\t"+str(xTest.shape))
print("yTest:\t"+str(yTest.shape))

print("---------------------------------------------")

# Adding the third dimension needed to use LSTM (samples, timestamps, features)
xtrain = np.reshape(xTrain.values, (xTrain.shape[0], xTrain.shape[1], 1))
ytrain = np.reshape(yTrain.values, (yTrain.shape[0], yTrain.shape[1], 1))
xval = np.reshape(xVal.values, (xVal.shape[0], xVal.shape[1], 1))
yval = np.reshape(yVal.values, (yVal.shape[0], yVal.shape[1], 1))
xtest = np.reshape(xTest.values, (xTest.shape[0], xTest.shape[1], 1))
ytest = np.reshape(yTest.values, (yTest.shape[0], yTest.shape[1], 1))

# New shapes after third dimension added
print("Dimensions afther third dimension added:")
print("xTrain:\t"+str(xtrain.shape))
print("yTrain:\t"+str(ytrain.shape))
print("xVal:\t"+str(xval.shape))
print("yVal:\t"+str(yval.shape))
print("xTest:\t"+str(xtest.shape))
print("yTest:\t"+str(ytest.shape))

DataFrame Preproccessed:
          col_0       col_1       col_2       col_3       col_4       col_5  \
0     96.691833  193.486833  700.986333  587.718333  454.512500  631.489833   
1    849.077500  746.732500  850.424667  745.088500  755.706667  688.645000   
2    202.375667  166.977000  403.916667  340.241000  649.179667  708.682500   
3    649.432333  665.403833  703.733167  438.760333  198.212667  114.285000   
4    109.810167  152.215167   84.124833  134.679500  183.520167  280.223333   
..          ...         ...         ...         ...         ...         ...   
725  360.369333  166.942500  390.007500  377.754500  226.036833  166.183833   
726  363.149000  464.485667  546.436500  617.035333  419.664667  571.180833   
727  571.733833  562.752667  548.686000  715.050000  446.255333  370.894167   
728  323.271333  320.158667  281.495667  465.665167  391.321167  609.922000   
729  467.921333  417.395833  578.181500  655.602167  524.690833  601.943333   

          col_6       col_

In [14]:
# Definicion del modelo
def construir_modelo(hp):
    model = keras.Sequential()
    # Primera capa creada manualmente. Se optimiza el numero de unidades
    model.add(LSTM(units=hp.Int('units', min_value=minUnits, max_value=maxUnits, step=stepsUnits, default=defaultUnits),
                   return_sequences=True, input_shape=(xtrain.shape[1], xtrain.shape[2])))
    # Se optimiza el numero de capas
    for i in range(hp.Int('num_layers', min_value=minLayers, max_value=maxLayers, default=defaultLayers)):
        # Para cada capa, se optimiza el numero de unidades de cada capa
        model.add(LSTM(units=hp.Int('units_' + str(i), min_value=minUnits, max_value=maxUnits, step=stepsUnits),
                       return_sequences=True))
        # Por cada capa, se optimiza el porcentaje de dropout
        model.add(
            Dropout(hp.Float('dropout_' + str(i), min_value=minDropout, max_value=maxDropout, default=defaultDropout)))
    # Se crea la ultima capa LSTM de forma manual, necesaria para no devolver la secuencia de datos completa
    # Se optimiza el numero de unidades de la capa
    model.add(LSTM(units=hp.Int('units_' + str(i + 1), min_value=minUnits, max_value=maxUnits, step=stepsUnits)))
    # Droppout optimizando el porcentaje
    model.add(
        Dropout(hp.Float('dropout_' + str(i + 1), min_value=minDropout, max_value=maxDropout, default=defaultDropout)))
    # Capa fully-connected de salida
    model.add(Dense(units=H))

    # Compilar modelo, optimizando learning_rate
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=lr)), loss='mean_squared_error',
                  metrics=[keras.metrics.MAPE, keras.metrics.MSE])
    # Devolver el modelo
    return model

In [15]:
# Directorio de trabajo y nombre del proyecto (dd-mm-aaaa)
workingDir = datetime.datetime.fromtimestamp(time.time()).strftime('%d-%m-%Y %hh:%mm')
projectName = "lstm"
# Crear optimizador
lstmOptimizer = RandomSearch(construir_modelo, objective=obje, directory=workingDir, project_name=projectName, max_trials=trials, executions_per_trial=executions)

# Imprimir resumen del espacio de busqueda
lstmOptimizer.search_space_summary()

# Ejecutar la busqueda determinando las epocas que se ejecutan
tiempo_inicial = time.time()
lstmOptimizer.search(x=xtrain, y=ytrain, verbose=0, epochs=searchEpochs, batch_size=batch, validation_data=(xval, yval))
tiempo_final = time.time()

# Mostrar resumen de los resultados
lstmOptimizer.results_summary()

# Almacenar en models los mejores modelos
model = lstmOptimizer.get_best_models(num_models=numBestModels)[0]

# Definir callBacks para guardar metricas y las epochas y modelos en cada una de las epocas
os.mkdir(workingDir+"/models")
logger = keras.callbacks.CSVLogger(workingDir+'/training.log')
modelSaver = keras.callbacks.ModelCheckpoint(filepath=workingDir+"/models/weights.{epoch:02d}.hdf5", monitor='val_mean_absolute_percentaje_error', verbose=0, save_best_only=False,
                                             save_weights_only=False, mode='auto', save_freq=1)

#Seguir entrenando el modelo hasta completar todas las epocas
history = model.fit(x=xtrain, y=ytrain, epochs=epch, initial_epoch=searchEpochs, batch_size=batch, verbose=0, validation_data=(xval, yval), callbacks=[logger,modelSaver])

# Mostrar información del modelo
print(history.history.keys())
print(model.history.history)
print(model.history.params)

InvalidArgumentError: Failed to create a directory: 27-12-2023 Dech:12m; Invalid argument

In [ ]:
# Graficas descenso de errores vs epocas
plot_history(model.history.history)
plt.show()

# Evaluar el modelo
results = model.evaluate(xtest, ytest, batch_size=batch, verbose=0)
print('test loss, test MAPE, tests MSE:', results)# Predecir el conjunto de test usando el mejor modelo
predicciones = model.predict(xtest, batch_size=batch, verbose=0)
predicciones = predicciones.flatten()

real=ytest.flatten()

#Comprobar que las metricas de las predicciones son iguales que la evaluacion del modelo
print("MAPE: ",keras.metrics.MAPE(real, predicciones))
print("MSE: ", keras.metrics.MSE(real, predicciones))

# Se deshace el reescalado de los datos
mreal = scaler.inverse_transform(real.reshape(1, -1)).flatten()
mpred = scaler.inverse_transform(predicciones.reshape(1, -1)).flatten()

# Imprimir métricas con datos desnormalizados
print("MAPE: ",keras.metrics.MAPE(mreal, mpred))
print("MSE: ", keras.metrics.MSE(mreal, mpred))# Graficar datos reales vs predichos de un muestreo de datos para apreciar mejor
sub_Real = mreal[:2000]
sub_Pred = mpred[:2000]
plt.figure(figsize = (15, 5))
plt.plot(sub_Real, label = "Real")
plt.plot(sub_Pred, label = "Predicción")
plt.xlabel("Tiempo")
plt.ylabel("Consumo eléctrico (Kw/h)")
plt.title("Serie temporal consumo eléctrico")
plt.legend()

plt.show()# Imprimir metricas de error
MRE = (1/(len(mreal)))*np.sum(abs(mpred-mreal)/mreal)
print("MRE:\t"+str(MRE*100))
print("MAE:\t"+str(metrics.mean_absolute_error(mreal,mpred)))
print("MSE:\t"+str(metrics.mean_squared_error(mreal,mpred)))

print("Tiempo de busqueda (en segundos):\t"+str(tiempo_final - tiempo_inicial))
print("Tiempo de busqueda (en horas):\t"+str((tiempo_final - tiempo_inicial)/3600))